# Approach - 2 : Two-step Flow

This approach involves first capturing all the details from the transcription and then formatting it in another LLM call.

### Why?
- Less Lossy
- More deterministic
- Lesser chances of hallucinations
- Highlight datapoints for follow-up/human inputs

In [51]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from typing import List, Optional, Dict
from datetime import date

load_dotenv()

llm = ChatOpenAI(
    name="Summarization", model="gpt-4o-mini", temperature=0.2, stream_usage=True
)

In [52]:
summary_instructions = """You are a clinical scribe tasked with documenting doctor-patient interactions into structured clinical notes suitable for downstream use in electronic health records, coding, and care planning.

Given a transcript and visit metadata, produce a clinical note that follows this structure:

**Chief Complaint (CC)**  
- Capture the patient's main concern in their own words, if available.

**History of Present Illness (HPI)**  
- Include onset, duration, location, quality, severity, modifying factors, associated symptoms, and relevant context.
- Extract weight change, medication adherence, functional impact, or other longitudinal details.

**Review of Systems (ROS)**  
- Include positive and negative findings by organ system if mentioned.

**Vital Signs**  
- Extract any stated values for: BP, HR, RR, Temp, SpO2, BMI, weight, and O2 delivery (e.g., on room air).

**Physical Exam**  
- Capture exam findings organized by system (general, CV, resp, neuro, skin, etc.).
- Use standard phrasing where possible.

**Medications**  
- List medications with name, dose, frequency, route, and indication if stated.

**Social History**  
- Capture living situation, work status, transportation, housing, financial stressors, and any other social determinants.

**Assessment & Plan**  
- Include diagnosis reasoning and impression. State each diagnosis with status (e.g., stable, worsening), and severity (e.g., mild/moderate/severe).
- Include medication changes (start, stop, change dose), referrals, diagnostic orders, follow-up, and patient education.

**Billing & Complexity**  
- If visit complexity, CPT codes, or ICD-10 codes are mentioned, include them.

**Instructions:**
- Use clinical phrasing, not raw transcript text.
- Clarify temporal or ambiguous phrases (e.g., "some time ago" → "~2 weeks ago") if possible.
- Don't invent information. Only extract what is clearly present.
- If a section is not mentioned, omit it.
- Be exhaustive where information is present.

Your goal is to create a clear, complete medical document that enables downstream structured data extraction and clinician review.
"""

user_input = """Dr. Harris: Good morning, Mr. Garcia. How have you been feeling since our last visit?
José Garcia: Morning, doc. I've been feeling okay overall, but lately my right foot has started burning and tingling a lot, especially when I get up after sitting.
Dr. Harris: I understand. Let me quickly confirm some things before we dive deeper. You're 67 years old, retired from construction, and living alone, correct?
José Garcia: Yes, that's right.
Dr. Harris: And you still have type 2 diabetes, high blood pressure, high cholesterol, and stage 3 chronic kidney disease. Am I missing anything?
José Garcia: No, you got it all. And the diabetes has been there for about 15 years now.
Dr. Harris: Got it. Any changes recently in your diet or your daily routine?
José Garcia: Honestly, transportation is still tough, and sometimes I skip meals because it's hard getting to the grocery store. Money can be tight, you know.
Dr. Harris: I completely understand; thanks for letting me know. This matters a lot for your overall health. How about medications—still on Metformin, Lisinopril, Atorvastatin, Gabapentin, and Aspirin?
José Garcia: Yeah, exactly those. Haven't missed doses, but the Gabapentin doesn't seem to help with the foot pain anymore.
Dr. Harris: Alright. Nurse Kelly will take your vitals quickly, then we'll check out your foot.
(Nurse Kelly takes vital signs.)
Nurse Kelly: Your blood pressure is 148 over 88, heart rate 80, oxygen is good at 96%, and your weight today is 82 kilos.
Dr. Harris: Your blood pressure is a bit high today, José. We'll keep an eye on that. How's your blood sugar management lately?
José Garcia: It's not great, honestly. The last A1C was about 8.2%. I admit, I've had some trouble controlling it.
Dr. Harris: Thanks for being honest about that. Let's check your feet. (Examines feet) Hmm, your pulses are a bit weak, and your sensation is reduced in your toes. There's no open wound, but there are some calluses.
José Garcia: Is that what's causing the burning feeling?
Dr. Harris: Yes, it's likely due to diabetic neuropathy. This happens when high sugars damage nerves over time. I'll prescribe Duloxetine—it can help manage nerve pain more effectively.
José Garcia: Sounds good, doc. Anything else I should do?
Dr. Harris: Yes, I'd like you to see a podiatrist. They'll manage those calluses and teach you foot care. Also, I want to involve our care manager, Mrs. Lee. She can help you access better transportation and some meal support programs.
(Mrs. Lee, care manager, joins briefly.)
Mrs. Lee: Hi José! I'll call you each week to see how your blood sugars are doing and help arrange transportation and meals. Would that be helpful?
José Garcia: Yes, thank you! I really appreciate it.
Dr. Harris: Excellent. Let's also give you the pneumonia vaccine today. We'll re-check labs next month. Does that sound good?
José Garcia: Sure, thanks again, doc.
"""

In [ ]:
summary_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", summary_instructions),
        ("user", "{input}"),
    ]
)

In [54]:
chunks = []
final_response = ""
for chunk in llm.stream(summary_prompt.format(input=user_input)):
    chunks.append(chunk)
    final_response += chunk.content
    print(chunk.content, end="", flush=True)

**Chief Complaint (CC)**  
- "My right foot has started burning and tingling a lot, especially when I get up after sitting."

**History of Present Illness (HPI)**  
- José Garcia is a 67-year-old male with a history of type 2 diabetes for approximately 15 years, hypertension, hyperlipidemia, and stage 3 chronic kidney disease. He reports a recent onset of burning and tingling in his right foot, particularly after prolonged sitting. He has experienced difficulty with transportation, leading to occasional missed meals due to challenges accessing the grocery store, which he attributes to financial constraints. His blood sugar management has been suboptimal, with a recent A1C of 8.2%. He has been adherent to his medications but notes that Gabapentin has not been effective for his foot pain.

**Review of Systems (ROS)**  
- **Endocrine:** Poor blood sugar control (A1C 8.2%).
- **Neurological:** Burning and tingling in the right foot, reduced sensation in toes.
- **Cardiovascular:** Hyperten

In [ ]:
# chunks = []
# output = ""
# display_handle = display(Markdown(""), display_id=True)

# for chunk in llm.stream(summary_prompt.format(input=user_input)):
#     chunks.append(chunk)
#     output += chunk.content
#     display_handle.update(Markdown(output))

**Chief Complaint (CC)**  
- "Lately my right foot has started burning and tingling a lot, especially when I get up after sitting."

**History of Present Illness (HPI)**  
- José Garcia is a 67-year-old male with a history of type 2 diabetes for approximately 15 years, high blood pressure, high cholesterol, and stage 3 chronic kidney disease. He reports new onset burning and tingling in his right foot, particularly exacerbated after prolonged sitting. He has been experiencing difficulty with transportation, which has led to occasional missed meals due to challenges in accessing the grocery store. Financial constraints have also been a concern. His diabetes management has been suboptimal, with a recent A1C of 8.2%. He reports adherence to his medications but notes that Gabapentin has not been effective for his foot pain.

**Review of Systems (ROS)**  
- **Endocrine:** Reports difficulty controlling blood sugar levels.  
- **Neurological:** Reports burning and tingling in the right foot, reduced sensation in toes.  
- **Cardiovascular:** No chest pain or palpitations reported.  
- **Musculoskeletal:** Reports calluses on feet.  
- **General:** No weight loss or gain reported; financial stressors mentioned.

**Vital Signs**  
- Blood Pressure: 148/88 mmHg  
- Heart Rate: 80 bpm  
- SpO2: 96% on room air  
- Weight: 82 kg  

**Physical Exam**  
- **General:** Alert and oriented, in no acute distress.  
- **Cardiovascular:** Pulses in feet are weak.  
- **Neurological:** Reduced sensation in toes.  
- **Dermatological:** Presence of calluses on feet, no open wounds noted.

**Medications**  
- Metformin: Dose not specified, frequency not specified, route not specified, indication: type 2 diabetes.  
- Lisinopril: Dose not specified, frequency not specified, route not specified, indication: hypertension.  
- Atorvastatin: Dose not specified, frequency not specified, route not specified, indication: hyperlipidemia.  
- Gabapentin: Dose not specified, frequency not specified, route not specified, indication: neuropathic pain.  
- Aspirin: Dose not specified, frequency not specified, route not specified, indication: cardiovascular protection.  
- Duloxetine: New prescription to be initiated for neuropathic pain.

**Social History**  
- Lives alone, retired from construction.  
- Reports difficulty with transportation and financial constraints affecting meal access.

**Assessment & Plan**  
1. **Diabetic Neuropathy** - Worsening symptoms (burning and tingling in the right foot).  
   - Plan: Initiate Duloxetine for neuropathic pain management.  
   - Refer to podiatrist for callus management and foot care education.  
2. **Uncontrolled Type 2 Diabetes** - A1C of 8.2%.  
   - Plan: Involve care manager (Mrs. Lee) for support with transportation and meal programs.  
   - Reassess blood sugar management strategies at next visit.  
3. **Hypertension** - Blood pressure 148/88 mmHg.  
   - Plan: Monitor blood pressure closely; consider medication adjustment if persistently elevated.  
4. **Pneumonia Vaccination** - Administer vaccine today.  
5. **Follow-up** - Schedule follow-up visit in one month for lab re-evaluation and assessment of diabetes management.

**Billing & Complexity**  
- Complexity of visit noted due to multiple chronic conditions and new medication management.  
- Potential CPT codes: 99214 (established patient visit, moderate complexity).  
- ICD-10 codes: E11.9 (Type 2 diabetes without complications), I10 (Essential hypertension), N18.3 (Stage 3 chronic kidney disease), G63.2 (Diabetic neuropathy).

### Now, Extraction Chain

*Pydantic Schema*

In [41]:
class VitalSigns(BaseModel):
    blood_pressure_systolic: Optional[int] = Field(
        None, description="Systolic BP in mmHg"
    )
    blood_pressure_diastolic: Optional[int] = Field(
        None, description="Diastolic BP in mmHg"
    )
    heart_rate: Optional[int] = Field(None, description="Heart rate in bpm")
    respiratory_rate: Optional[int] = Field(
        None, description="Respiratory rate per minute"
    )
    temperature: Optional[float] = Field(None, description="Temperature in Fahrenheit")
    oxygen_saturation: Optional[int] = Field(None, description="SpO2 percentage")
    oxygen_saturation_on_room_air: Optional[bool] = Field(
        None, description="Whether O2 sat is on room air"
    )
    weight_kg: Optional[float] = Field(None, description="Weight in kilograms")
    bmi: Optional[float] = Field(None, description="Body Mass Index")
    weight_change: Optional[str] = Field(
        None, description="Weight change from previous visit"
    )


class Medication(BaseModel):
    name: str = Field(..., description="Medication name")
    dosage: str = Field(..., description="Dosage amount")
    frequency: str = Field(..., description="Frequency (e.g., BID, daily, TID)")
    route: Optional[str] = Field(None, description="Route of administration")


class SocialHistory(BaseModel):
    living_situation: Optional[str] = Field(None, description="Who patient lives with")
    occupation: Optional[str] = Field(None, description="Current or former occupation")
    employment_status: Optional[str] = Field(None, description="Working, retired, etc.")
    transportation_barriers: Optional[bool] = Field(
        None, description="Has transportation difficulties"
    )
    food_insecurity: Optional[bool] = Field(
        None, description="Experiences food insecurity"
    )
    financial_strain: Optional[bool] = Field(
        None, description="Has financial difficulties"
    )
    housing_stability: Optional[str] = Field(None, description="Housing situation")


class PhysicalExam(BaseModel):
    general_appearance: Optional[str] = Field(None, description="General appearance")
    cardiovascular: Optional[str] = Field(
        None, description="Cardiovascular exam findings"
    )
    respiratory: Optional[str] = Field(None, description="Respiratory exam findings")
    extremities: Optional[str] = Field(None, description="Extremity exam findings")
    neurological: Optional[str] = Field(None, description="Neurological exam findings")
    skin: Optional[str] = Field(None, description="Skin exam findings")
    musculoskeletal: Optional[str] = Field(None, description="Musculoskeletal findings")
    other_findings: Optional[str] = Field(
        None, description="Other examination findings"
    )


class Diagnosis(BaseModel):
    condition: str = Field(..., description="Diagnosis or condition name")
    icd10_code: Optional[str] = Field(None, description="ICD-10 diagnosis code")
    status: Optional[str] = Field(None, description="Active, stable, worsening, etc.")
    severity: Optional[str] = Field(None, description="Mild, moderate, severe")


class Medication_Plan(BaseModel):
    medication: str = Field(..., description="Medication name")
    action: str = Field(
        ..., description="start, increase, decrease, discontinue, continue"
    )
    dosage: str = Field(..., description="New or current dosage")
    frequency: str = Field(..., description="Dosing frequency")
    indication: Optional[str] = Field(None, description="What condition this treats")


class Order(BaseModel):
    type: str = Field(..., description="lab, imaging, procedure, etc.")
    description: str = Field(..., description="Specific test or procedure ordered")
    indication: Optional[str] = Field(None, description="Clinical reason for order")
    timing: Optional[str] = Field(None, description="When to complete")


class Referral(BaseModel):
    specialty: str = Field(..., description="Type of specialist or service")
    reason: str = Field(..., description="Reason for referral")
    urgency: Optional[str] = Field(None, description="Routine, urgent, stat")


class CareCoordination(BaseModel):
    care_manager_involved: Optional[bool] = Field(
        None, description="Care manager participated"
    )
    care_manager_name: Optional[str] = Field(None, description="Name of care manager")
    services_coordinated: Optional[List[str]] = Field(
        None, description="Services arranged"
    )
    home_health_ordered: Optional[bool] = Field(
        None, description="Home health services ordered"
    )
    social_services_referral: Optional[bool] = Field(
        None, description="Social services involved"
    )


class FollowUp(BaseModel):
    timing: str = Field(
        ..., description="When to follow up (e.g., '2 weeks', '1 month')"
    )
    location: Optional[str] = Field(None, description="Clinic, telehealth, etc.")
    purpose: Optional[str] = Field(None, description="Reason for follow-up visit")


class BillingInfo(BaseModel):
    cpt_code: str = Field(..., description="CPT code for visit")
    visit_complexity: Optional[str] = Field(
        None, description="Low, moderate, high complexity"
    )
    icd10_codes: List[str] = Field(..., description="All ICD-10 codes for billing")


class Subjective(BaseModel):
    patient_name: Optional[str] = Field(None, description="Patient name")
    age: Optional[int] = Field(None, description="Patient age")
    gender: Optional[str] = Field(None, description="Patient gender")
    ethnicity: Optional[str] = Field(None, description="Patient ethnicity")
    chief_complaint: str = Field(..., description="Primary reason for visit")
    history_present_illness: str = Field(..., description="Detailed HPI narrative")
    medical_history: List[str] = Field(
        default_factory=list, description="Past medical history"
    )
    surgical_history: Optional[List[str]] = Field(
        None, description="Past surgical history"
    )
    family_history: Optional[str] = Field(None, description="Relevant family history")
    social_history: Optional[SocialHistory] = Field(
        None, description="Social history details"
    )
    current_medications: List[Medication] = Field(
        default_factory=list, description="Current medications"
    )
    allergies: Optional[List[str]] = Field(None, description="Known allergies")
    review_of_systems: Optional[str] = Field(
        None, description="Additional symptoms mentioned"
    )


class Objective(BaseModel):
    vital_signs: Optional[VitalSigns] = Field(
        None, description="Vital sign measurements"
    )
    physical_exam: Optional[PhysicalExam] = Field(
        None, description="Physical examination findings"
    )
    diagnostic_results: Optional[Dict[str, str]] = Field(
        None, description="Lab or test results mentioned"
    )


class Assessment(BaseModel):
    primary_diagnoses: List[Diagnosis] = Field(
        ..., description="Primary diagnoses in order of priority"
    )
    secondary_diagnoses: Optional[List[Diagnosis]] = Field(
        None, description="Secondary/chronic diagnoses"
    )
    social_determinants: Optional[List[str]] = Field(
        None, description="Social factors affecting health"
    )
    clinical_impression: Optional[str] = Field(
        None, description="Overall clinical assessment"
    )


class Plan(BaseModel):
    medications: Optional[List[Medication_Plan]] = Field(
        None, description="Medication changes"
    )
    orders: Optional[List[Order]] = Field(None, description="Diagnostic orders")
    referrals: Optional[List[Referral]] = Field(
        None, description="Specialist referrals"
    )
    care_coordination: Optional[CareCoordination] = Field(
        None, description="Care team coordination"
    )
    patient_education: Optional[List[str]] = Field(
        None, description="Education provided"
    )
    follow_up: Optional[FollowUp] = Field(None, description="Follow-up plans")
    preventive_care: Optional[List[str]] = Field(
        None, description="Vaccines, screenings provided"
    )


class SOAPNote(BaseModel):
    """
    Complete SOAP note structure for clinical documentation
    """

    visit_date: Optional[date] = Field(None, description="Date of visit")
    subjective: Subjective = Field(..., description="Subjective section")
    objective: Objective = Field(..., description="Objective section")
    assessment: Assessment = Field(..., description="Assessment section")
    plan: Plan = Field(..., description="Plan section")
    billing_info: Optional[BillingInfo] = Field(None, description="Billing information")

    class Config:
        json_encoders = {date: lambda v: v.isoformat()}
        json_schema_extra = {
            "example": {
                "visit_date": "2024-06-14",
                "subjective": {
                    "patient_name": "José Garcia",
                    "age": 67,
                    "gender": "male",
                    "ethnicity": "Hispanic",
                    "chief_complaint": "Burning and tingling pain in right foot",
                    "history_present_illness": "67-year-old male reports burning and tingling in right foot, especially when standing after prolonged sitting. Symptoms have worsened recently.",
                    "medical_history": [
                        "Type 2 diabetes (15 years)",
                        "Hypertension",
                        "Hyperlipidemia",
                        "CKD Stage 3",
                    ],
                    "social_history": {
                        "living_situation": "Lives alone",
                        "occupation": "Retired construction worker",
                        "transportation_barriers": True,
                        "food_insecurity": True,
                    },
                },
                "assessment": {
                    "primary_diagnoses": [
                        {
                            "condition": "Diabetic peripheral neuropathy",
                            "icd10_code": "E11.40",
                            "status": "worsening",
                        }
                    ]
                },
                "billing_info": {
                    "cpt_code": "99214",
                    "icd10_codes": ["E11.40", "I10", "N18.3", "Z59.4"],
                },
            }
        }

In [58]:
llm_with_structured_output = ChatOpenAI(
    name="Extraction", model="gpt-4o-mini", temperature=0
).with_structured_output(schema=SOAPNote)

extraction_instructions = """You are a clinical scribe. Given this clinical note summary, generate a highly accurate SOAP-formatted EHR, capturing all minute details."""

extraction_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", extraction_instructions),
        ("user", "{input}"),
    ]
)

In [61]:
response = llm_with_structured_output.invoke(
    summary_prompt.format(input=final_response)
)

In [66]:
print(response.model_dump_json(indent=4))

{
    "visit_date": "2024-06-14",
    "subjective": {
        "patient_name": "José Garcia",
        "age": 67,
        "gender": "male",
        "ethnicity": "Hispanic",
        "chief_complaint": "My right foot has started burning and tingling a lot, especially when I get up after sitting.",
        "history_present_illness": "José Garcia is a 67-year-old male with a history of type 2 diabetes for approximately 15 years, hypertension, hyperlipidemia, and stage 3 chronic kidney disease. He reports a recent onset of burning and tingling in his right foot, particularly after prolonged sitting. He has experienced difficulty with transportation, leading to occasional missed meals due to challenges accessing the grocery store, which he attributes to financial constraints. His blood sugar management has been suboptimal, with a recent A1C of 8.2%. He has been adherent to his medications but notes that Gabapentin has not been effective for his foot pain.",
        "medical_history": [
       